In [1]:
# Dependencies
from bs4 import BeautifulSoup 
import requests
import pandas as pd
from splinter import Browser
from selenium import webdriver
import time

In [2]:
#set up path for windows
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url="https://www.espn.com/college-football/teams"
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html, 'lxml')
browser.quit()

In [3]:
stat_link=soup.find_all('a', class_="AnchorLink")
team={
    "name":[],
    "url":[]
}
name_pars=""


for alink in stat_link:
    if alink.text=="Stats":
        length=0
        team["url"].append("https://www.espn.com"+alink.get("href"))
        name_pars=alink.get("href").split('/')
        length=len(name_pars)
        length=length-1
        team["name"].append(name_pars[length])
                


In [4]:
df=pd.DataFrame(team)
df=df[df.name != 'stats']
df.head()

,name,url
1,cincinnati-bearcats,https://www.espn.com/college-football/team/sta...
2,east-carolina-pirates,https://www.espn.com/college-football/team/sta...
3,houston-cougars,https://www.espn.com/college-football/team/sta...
4,memphis-tigers,https://www.espn.com/college-football/team/sta...
5,navy-midshipmen,https://www.espn.com/college-football/team/sta...


In [5]:
def get_stat_tables(url):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)

    browser.visit(url)
    html=browser.html
    browser.quit()
    return(pd.read_html(html))
    

In [6]:
df["soap"] = df["url"].map(get_stat_tables)

In [7]:
def stats_table_merge(stats_tables):
    passing=pd.merge(stats_tables[0], stats_tables[1],  left_index=True, right_index=True)
    rushing=pd.merge(stats_tables[2], stats_tables[3],  left_index=True, right_index=True)
    receiving=pd.merge(stats_tables[4], stats_tables[5],  left_index=True, right_index=True)
    team_data={ "passing":passing.to_json(),
        "rushing":rushing.to_json(),
        "receiving":receiving.to_json()
         }
    return(team_data)

In [8]:
df["stats_tables"]=df["soap"].map(stats_table_merge)

In [9]:
newdf=df[{"name", "stats_tables"}]
newdf.set_index("name", inplace=True)


,stats_tables
name,
cincinnati-bearcats,"{'passing': '{""Name"":{""0"":""Desmond Ridder QB"",..."
east-carolina-pirates,"{'passing': '{""Name"":{""0"":""Holton Ahlers QB"",""..."
houston-cougars,"{'passing': '{""Name"":{""0"":""D'Eriq King QB"",""1""..."
memphis-tigers,"{'passing': '{""Name"":{""0"":""Brady White QB"",""1""..."
navy-midshipmen,"{'passing': '{""Name"":{""0"":""Malcolm Perry QB"",""..."


In [10]:
import pymongo
import json
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.ncaa_football_stats
collection = db.players

records = json.loads(newdf.to_json()).values()
collection.insert(records)

C:\Users\kevin\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


[ObjectId('5da7a2688d050381d8ead922')]